<div class='alert-block alert-info'>
    <br>
    <h1 align="center"> AI Project 
    <h3 align="center">Nadia Godje </h3>
    <h3 align="center">Stive </h3>
    <\br>
</div>

Pour commencer, nous devons charger les données des défauts à partir du fichier CSV "defects.csv" et définir les informations sur les biscuits

In [ ]:
import pandas as pd

# Chargement des données des défauts depuis le fichier CSV
def load_defects_data():
    defects_data = pd.read_csv('defects.csv')
    return defects_data

defects_data = load_defects_data()

biscuits_info = [
    {'num':0,'length': 4, 'value': 6, 'defauts': {'a': 4, 'b': 2, 'c': 3}},  # Biscuit 0
    {'num':1,'length': 8, 'value': 12, 'defauts': {'a': 5, 'b': 4, 'c': 4}},  # Biscuit 1
    {'num':2,'length': 2, 'value': 1, 'defauts': {'a': 1, 'b': 2, 'c': 1}},  # Biscuit 2
    {'num':3,'length': 5, 'value': 8, 'defauts': {'a': 2, 'b': 3, 'c': 2}}   # Biscuit 3
]

roll_length = 500

print(biscuits_info)
print(roll_length)
print(defects_data.head())


[{'num': 0, 'length': 4, 'value': 6, 'defauts': {'a': 4, 'b': 2, 'c': 3}}, {'num': 1, 'length': 8, 'value': 12, 'defauts': {'a': 5, 'b': 4, 'c': 4}}, {'num': 2, 'length': 2, 'value': 1, 'defauts': {'a': 1, 'b': 2, 'c': 1}}, {'num': 3, 'length': 5, 'value': 8, 'defauts': {'a': 2, 'b': 3, 'c': 2}}]
500
            x class
0  355.449335     c
1   92.496236     a
2  141.876795     c
3  431.833902     c
4  435.028461     c


Ce code charge les informations sur les biscuits et la longueur du rouleau, puis charge les données des défauts à partir du fichier CSV. Il affiche également les lignes des données des défauts que les biscuits peuvent supporter

Après avoir chargé les données des défauts et défini les informations sur les biscuits, nous pouvons commencer à formuler l'algorithme pour placer les biscuits sur le rouleau tout en respectant les contraintes spécifiées. 

**Greedy-search**

La fonction find_valid_positions cherche pour chaque biscuit la meilleure position possible sur le rouleau en considérant les contraintes de chevauchement et les seuils de défauts. Elle itère sur chaque position possible pour un biscuit, vérifie s'il y a un chevauchement avec d'autres biscuits déjà placés et si le seuil de défauts est respecté.

Puis, la fonction calculate_total_value calcule la **valeur totale des biscuits placés** (ce qu'on cherche à optimiser).

Afin de maximiser la valeur totale des biscuits en limitant le cout en temps et en calcul, nous avons demandé au l'algorithme de ne placer que les biscuits 1 (valeur = 12) avec un nombre maximum de 110. Il a donc placé 424/8=53 biscuits pour une valeur totale de 636.

In [43]:
def find_valid_positions(biscuits_info, roll_length, defects_data, num_placements):
    total_length_placed = 0
    all_valid_positions = []

    biscuits_to_place = [biscuit for biscuit in biscuits_info if biscuit['num'] == 1]

    for biscuit in biscuits_to_place:
        valid_positions = []
        for placement in range(num_placements):
            best_positions = []
            best_value = 0

            for position in range(roll_length - biscuit['length'] + 1):
                overlap = False
                for placed_biscuit in valid_positions:
                    if position <= placed_biscuit['position'] < position + biscuit['length']:
                        overlap = True
                        break

                if not overlap:
                    defects_in_interval = defects_data[(defects_data['x'] >= position) &
                                                      (defects_data['x'] < position + biscuit['length'])]

                    valid_biscuit = True
                    for class_name, threshold in biscuit['defauts'].items():
                        class_defects = defects_in_interval[defects_in_interval['class'] == class_name]
                        if class_defects.shape[0] > threshold:
                            valid_biscuit = False
                            break

                    if valid_biscuit:
                        total_value = biscuit['value']
                        if best_positions:
                            total_value += best_value
                        if total_value > best_value:
                            best_value = total_value
                            best_positions = [{'biscuit_': biscuit['num'], 'position': position}]
                        elif total_value == best_value:
                            best_positions.append({'biscuit_': biscuit['num'], 'position': position})

            if best_positions:
                valid_positions.extend(best_positions)

        for pos in valid_positions:
            biscuit_id = pos['biscuit_']
            total_length_placed += biscuit['length']
        
        all_valid_positions.extend(valid_positions)

        if total_length_placed >= roll_length:
            break  # Stop if total length exceeds roll length

    return all_valid_positions, total_length_placed

def calculate_total_value(valid_positions, biscuits_info):
    total_value = 0
    
    for pos in valid_positions:
        biscuit_id = pos['biscuit_']
        total_value += biscuits_info[biscuit_id]['value']
    
    return total_value

# Utilisation de la fonction pour placer plusieurs fois le même biscuit
num_placements = 110  # Nombre de placements souhaités pour chaque biscuit
valid_positions, total_length = find_valid_positions(biscuits_info, roll_length, defects_data, num_placements)
total_biscuit_value = calculate_total_value(valid_positions, biscuits_info)

if valid_positions:
    print("Positions valides des biscuits placés:", valid_positions)
    print("Somme des longueurs des biscuits placés:", total_length)
    print("Valeur totale des biscuits placés:", total_biscuit_value)
else:
    print("Aucun biscuit 1 de ne peut être placé.")


Positions valides des biscuits placés: [{'biscuit_': 1, 'position': 492}, {'biscuit_': 1, 'position': 484}, {'biscuit_': 1, 'position': 474}, {'biscuit_': 1, 'position': 466}, {'biscuit_': 1, 'position': 458}, {'biscuit_': 1, 'position': 450}, {'biscuit_': 1, 'position': 442}, {'biscuit_': 1, 'position': 434}, {'biscuit_': 1, 'position': 425}, {'biscuit_': 1, 'position': 417}, {'biscuit_': 1, 'position': 409}, {'biscuit_': 1, 'position': 400}, {'biscuit_': 1, 'position': 392}, {'biscuit_': 1, 'position': 384}, {'biscuit_': 1, 'position': 375}, {'biscuit_': 1, 'position': 360}, {'biscuit_': 1, 'position': 347}, {'biscuit_': 1, 'position': 339}, {'biscuit_': 1, 'position': 330}, {'biscuit_': 1, 'position': 322}, {'biscuit_': 1, 'position': 314}, {'biscuit_': 1, 'position': 306}, {'biscuit_': 1, 'position': 298}, {'biscuit_': 1, 'position': 286}, {'biscuit_': 1, 'position': 278}, {'biscuit_': 1, 'position': 270}, {'biscuit_': 1, 'position': 262}, {'biscuit_': 1, 'position': 254}, {'biscui

**Recherche en largeur (BFS)**

Cette méthode va parcourir toutes les configurations possibles pour le placement des biscuits, en énumérant toutes les combinaisons et en gardant la meilleure configuration qui respecte les contraintes.

In [8]:
from queue import Queue

def evaluate_solution(solution, defects_data):
    # Cette fonction calcule la valeur totale de la solution en fonction des défauts
    total_value = sum(biscuit['biscuit']['value'] for biscuit in solution)
    return total_value

def breadth_first_search(biscuits_info, roll_length, defects_data):
    q = Queue()
    q.put([])  # Commence avec une solution vide

    best_solution = None
    best_value = 0
    visited = set()

    while not q.empty():
        current_solution = q.get()
        value = evaluate_solution(current_solution, defects_data)

        if value > best_value:
            best_value = value
            best_solution = current_solution

        # Étendre la solution actuelle avec de nouvelles positions pour les biscuits
        for biscuit in biscuits_info:
            for position in range(roll_length - biscuit['length'] + 1):
                new_biscuit_placement = {'biscuit': biscuit, 'position': position}
                new_solution = current_solution + [new_biscuit_placement]
                sorted_new_solution = tuple(sorted((bisc['biscuit']['num'], bisc['position']) for bisc in new_solution))

                if sorted_new_solution not in visited:
                    q.put(new_solution)
                    visited.add(sorted_new_solution)

    return best_solution, best_value

# Utilisation de l'algorithme de recherche en largeur pour résoudre le problème
best_solution, best_value = breadth_first_search(biscuits_info, roll_length, defects_data)

# Calculate additional information
num_placed_biscuits = len(best_solution) if best_solution else 0
total_length_of_biscuits = sum(biscuit['biscuit']['length'] for biscuit in best_solution) if best_solution else 0

# Affichage des résultats
print("Meilleure solution trouvée:")
if best_solution:
    for biscuit in best_solution:
        print(biscuit)
print("Nombre de biscuits placés:", num_placed_biscuits)
print("Somme de la longueur des biscuits:", total_length_of_biscuits)
print("Valeur de la meilleure solution:", best_value)

#BOUCLE INFINIIIIIIIIE


In [ ]:
from collections import deque

def intervals_overlap(start1, end1, start2, end2):
    return not (end1 <= start2 or start1 >= end2)

def can_place_biscuit(positions, lengths, biscuit_thresholds, defects_data):
    for position, length in zip(positions, lengths):
        defects_in_interval = defects_data[
            (defects_data['x'] >= position) &
            (defects_data['x'] < position + length)
        ]

        defects_count = {class_name: 0 for class_name in 'abc'}
        for class_name in 'abc':
            class_defects = defects_in_interval[defects_in_interval['class'] == class_name]
            defects_count[class_name] += class_defects.shape[0]

        for class_name, threshold in biscuit_thresholds.items():
            if defects_count[class_name] > threshold:
                return False
    return True

def breadth_first_search(biscuits_info, roll_length, defects_data):
    q = deque()
    q.append([])  # Commence avec une solution vide

    best_solution = None
    best_value = 0

    while q:
        current_solution = q.popleft()
        total_length = sum(biscuit['length'] for biscuit in current_solution)

        if total_length <= roll_length:
            value = sum(biscuit['value'] for biscuit in current_solution)
            if value > best_value:
                best_value = value
                best_solution = current_solution

            for biscuit in biscuits_info:
                for position in range(roll_length - biscuit['length'] + 1):
                    new_placement = {'biscuit': biscuit, 'position': position}
                    new_solution = current_solution + [new_placement]
                    if can_place_biscuit(
                        [b['position'] for b in new_solution],
                        [b['biscuit']['length'] for b in new_solution],
                        biscuit['defauts'],
                        defects_data
                    ):
                        q.append(new_solution)

    return best_solution, best_value

# Utilisation de l'algorithme de recherche en largeur pour résoudre le problème
best_solution, best_value = breadth_first_search(biscuits_info, roll_length, defects_data)

# Affichage des résultats
print("Meilleure solution trouvée:")
if best_solution:
    for biscuit in best_solution:
        print(biscuit)
print("Valeur de la meilleure solution:", best_value)


**Algo génétique**

In [22]:
import random

# Fonction pour initialiser une population aléatoire de solutions
def initialize_population(biscuits_info, roll_length, population_size):
    population = []
    for _ in range(population_size):
        solution = []
        for biscuit in biscuits_info:
            position = random.randint(0, roll_length - biscuit['length'])
            solution.append({'biscuit': biscuit, 'position': position})
        population.append(solution)
    return population

# Fonction pour évaluer la performance d'une solution
def evaluate_solution(solution, defects_data):
    total_value = sum(biscuit['biscuit']['value'] for biscuit in solution)
    total_length = sum(biscuit['biscuit']['length'] for biscuit in solution)
    
    # Vérifier les contraintes de défauts
    defects_count = {class_name: 0 for class_name in 'abc'}
    for biscuit in solution:
        defects_in_interval = defects_data[
            (defects_data['x'] >= biscuit['position']) &
            (defects_data['x'] < biscuit['position'] + biscuit['biscuit']['length'])
        ]
        for class_name in 'abc':
            class_defects = defects_in_interval[defects_in_interval['class'] == class_name]
            defects_count[class_name] += class_defects.shape[0]
    
    for class_name, threshold in solution[0]['biscuit']['defauts'].items():
        if defects_count[class_name] > threshold:
            return 0  # Retourne 0 si une contrainte est violée
    
    return total_value

# Fonction pour sélectionner les meilleurs parents dans la population
def select_parents(population, defects_data):
    scored_population = [(solution, evaluate_solution(solution, defects_data)) for solution in population]
    scored_population.sort(key=lambda x: x[1], reverse=True)
    return [solution for solution, _ in scored_population[:len(scored_population)//2]]

# Fonction pour réaliser le croisement entre deux parents
def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child = parent1[:crossover_point] + parent2[crossover_point:]
    return child

# Fonction pour réaliser la mutation sur une solution
def mutate(solution, roll_length):
    mutated_solution = solution.copy()
    index = random.randint(0, len(solution) - 1)
    biscuit = solution[index]
    biscuit['position'] = random.randint(0, roll_length - biscuit['biscuit']['length'])
    mutated_solution[index] = biscuit
    return mutated_solution

# Fonction principale de l'algorithme génétique
def genetic_algorithm(biscuits_info, roll_length, defects_data, population_size=100, generations=100):
    population = initialize_population(biscuits_info, roll_length, population_size)
    best_solution = None
    best_value = 0
    
    for _ in range(generations):
        parents = select_parents(population, defects_data)
        children = []
        
        while len(children) < population_size:
            parent1 = random.choice(parents)
            parent2 = random.choice(parents)
            child = crossover(parent1, parent2)
            if random.random() < 0.1:  # Probabilité de mutation
                child = mutate(child, roll_length)
            children.append(child)
        
        population = children
        
        for solution in population:
            value = evaluate_solution(solution, defects_data)
            if value > best_value:
                best_value = value
                best_solution = solution
    
    return best_solution, best_value

# Utilisation de l'algorithme génétique pour résoudre le problème
best_solution, best_value = genetic_algorithm(biscuits_info, roll_length, defects_data)
print("Meilleure solution trouvée:", best_solution)
print("Valeur de la meilleure solution:", best_value)


Meilleure solution trouvée: [{'biscuit': {'num': 0, 'length': 4, 'value': 6, 'defauts': {'a': 4, 'b': 2, 'c': 3}}, 'position': 417}, {'biscuit': {'num': 1, 'length': 8, 'value': 12, 'defauts': {'a': 5, 'b': 4, 'c': 4}}, 'position': 223}, {'biscuit': {'num': 2, 'length': 2, 'value': 1, 'defauts': {'a': 1, 'b': 2, 'c': 1}}, 'position': 279}, {'biscuit': {'num': 3, 'length': 5, 'value': 8, 'defauts': {'a': 2, 'b': 3, 'c': 2}}, 'position': 164}]
Valeur de la meilleure solution: 27


Pour placer plusieurs fois le même biscuit

In [24]:
import random

# Fonction pour initialiser une population aléatoire de solutions
def initialize_population(biscuits_info, roll_length, population_size):
    population = []
    for _ in range(population_size):
        solution = []
        for biscuit in biscuits_info:
            occurrences = random.randint(1, 5)  # Nombre aléatoire d'occurrences pour chaque biscuit (entre 1 et 5)
            for _ in range(occurrences):
                position = random.randint(0, roll_length - biscuit['length'])
                solution.append({'biscuit': biscuit, 'position': position})
        population.append(solution)
    return population

# Fonction pour évaluer la performance d'une solution
def evaluate_solution(solution, defects_data):
    total_value = sum(biscuit['biscuit']['value'] for biscuit in solution)
    total_length = sum(biscuit['biscuit']['length'] for biscuit in solution)
    
    # Vérifier les contraintes de défauts
    defects_count = {class_name: 0 for class_name in 'abc'}
    for biscuit in solution:
        defects_in_interval = defects_data[
            (defects_data['x'] >= biscuit['position']) &
            (defects_data['x'] < biscuit['position'] + biscuit['biscuit']['length'])
        ]
        for class_name in 'abc':
            class_defects = defects_in_interval[defects_in_interval['class'] == class_name]
            defects_count[class_name] += class_defects.shape[0]
    
    for class_name, threshold in solution[0]['biscuit']['defauts'].items():
        if defects_count[class_name] > threshold:
            return 0  # Retourne 0 si une contrainte est violée
    
    return total_value

# Fonction pour sélectionner les meilleurs parents dans la population
def select_parents(population, defects_data):
    scored_population = [(solution, evaluate_solution(solution, defects_data)) for solution in population]
    scored_population.sort(key=lambda x: x[1], reverse=True)
    
    return [solution for solution, _ in scored_population[:len(scored_population)//2]]

# Fonction pour réaliser le croisement entre deux parents
def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child = parent1[:crossover_point] + parent2[crossover_point:]

    return child

# Fonction pour réaliser la mutation sur une solution
def mutate(solution, roll_length):
    mutated_solution = solution.copy()
    index = random.randint(0, len(solution) - 1)
    biscuit = solution[index]
    biscuit['position'] = random.randint(0, roll_length - biscuit['biscuit']['length'])
    mutated_solution[index] = biscuit

    return mutated_solution


def genetic_algorithm(biscuits_info, roll_length, defects_data, population_size=100, generations=100):
    population = initialize_population(biscuits_info, roll_length, population_size)
    best_solution = None
    best_value = 0
    
    for _ in range(generations):
        parents = select_parents(population, defects_data)
        children = []
        
        while len(children) < population_size:
            parent1 = random.choice(parents)
            parent2 = random.choice(parents)
            child = crossover(parent1, parent2)
            if random.random() < 0.1:  # Probabilité de mutation
                child = mutate(child, roll_length)
            children.append(child)
        
        population = children
        
        for solution in population:
            value = evaluate_solution(solution, defects_data)
            if value > best_value:
                best_value = value
                best_solution = solution
    
    return best_solution, best_value

# Utilisation de l'algorithme génétique pour résoudre le problème
best_solution, best_value = genetic_algorithm(biscuits_info, roll_length, defects_data)
print("Meilleure solution trouvée:", best_solution)
print("Valeur de la meilleure solution:", best_value)


Meilleure solution trouvée: None
Valeur de la meilleure solution: 0


Suite à plusieurs tentatives après des nombres d'occurences différentes, notre algo génétique n'arrive à placer qu'une seule occurence de chaque biscuit, pour une valeur maximale de 27.

**CSP (Constraint Satisfaction Problem)**

Le CSP  est un bon moyen de modéliser ce problème pour trouver **les meilleures positions** pour placer les biscuits tout en **respectant les contraintes**. Pour cela, nous allons utiliser la **bibliothèque python-constraint**, qui permet de définir des variables, des domaines et des contraintes pour résoudre ce genre de problèmes.

Nous allons définir les variables et contraintes comme suit :

Variables : Chaque biscuit aura une variable représentant sa position sur le rouleau.
Contraintes :
Les positions doivent être des nombres entiers.
La somme des longueurs des biscuits placés doit être inférieure à la longueur du rouleau.
Aucun chevauchement n'est autorisé entre les biscuits.
Les défauts dans l'intervalle de chaque biscuit ne doivent pas dépasser les seuils spécifiés.

In [22]:
from constraint import Problem

def intervals_overlap(start1, end1, start2, end2):
    return not (end1 <= start2 or start1 >= end2)

def can_place_biscuit(position, length, defects_data, biscuit_thresholds):
    defects_in_interval = defects_data[(defects_data['x'] >= position) &
                                      (defects_data['x'] < position + length)]

    for class_name, threshold in biscuit_thresholds.items():
        class_defects = defects_in_interval[defects_in_interval['class'] == class_name]
        if class_defects.shape[0] > threshold:
            return False
    
    return True

def solve_biscuit_placement(biscuits_info, roll_length, defects_data):
    problem = Problem()

    # Define variables (positions of biscuits on the roll)
    for i, biscuit in enumerate(biscuits_info):
        problem.addVariable(f"biscuit_{i}_position", range(roll_length - biscuit['length'] + 1))

    # Add constraints to avoid overlap and check defects
    for i, biscuit in enumerate(biscuits_info):
        biscuit_length = biscuit['length']
        biscuit_thresholds = biscuit['defauts']
        for j, other_biscuit in enumerate(biscuits_info):
            if i != j:
                other_biscuit_length = other_biscuit['length']
                problem.addConstraint(
                    lambda pos1, pos2, len1=biscuit_length, len2=other_biscuit_length: 
                    not intervals_overlap(pos1, pos1 + len1, pos2, pos2 + len2),
                    (f"biscuit_{i}_position", f"biscuit_{j}_position")
                )

        problem.addConstraint(
            lambda pos, length=biscuit_length, thresholds=biscuit_thresholds: 
            can_place_biscuit(pos, length, defects_data, thresholds),
            (f"biscuit_{i}_position",)
        )

    total_value = 0
    solutions = problem.getSolutionIter()
    for solution in solutions:
        print(solution)
        # Calculate the total value of placed biscuits
        placed_biscuits = [biscuits_info[int(key.split('_')[1])]['value'] for key, value in solution.items()]
        total_value += sum(placed_biscuits)
    
    return total_value

# Utilisation de la fonction pour résoudre le problème de placement des biscuits
for solution in solve_biscuit_placement(biscuits_info, roll_length, defects_data):
    print(solution)
total_biscuit_value = solve_biscuit_placement(biscuits_info, roll_length, defects_data)
print(f"La somme des valeurs des biscuits placés est : {total_biscuit_value}")
